In [1]:
!pip install -q -U transformers datasets accelerate evaluate peft huggingface_hub bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import pandas as pd
import os 
import re

import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset, DatasetDict
import torch

2025-10-03 01:38:33.181904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759455513.370905      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759455513.427049      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load data and preprocess

In [3]:
import pandas as pd
CSV_PATH = "/kaggle/input/dsc-2025-llm-hallucination/final_train.csv"  
LABEL_COL = "label"    
TEXT_COLS = ["context", "prompt", "response"]  
VAL_SIZE = 0.2
SEED = 42

In [4]:
df = pd.read_csv(CSV_PATH)
df.head()

,id,context,prompt,response,label,new_context,final_context
0,9b1ea51d-d1ff-45ba-8cf1-6a91328e8600,"Vào những năm 1870, hai nhà điêu khắc Augustus...","Vào những năm 1960, nơi nào trở thành trung tâ...",Quảng trường Washington là trung tâm của thế h...,extrinsic,"Vào những năm 1960, khu này trở thành một tron...","Vào những năm 1960, khu này trở thành một tron..."
1,db7a89c6-2a6a-42af-beef-58e557ecc819,Cách mạng Tháng Mười đã biến một cuộc chiến tr...,Cách mạng Tháng Mười đã khởi xướng chủ nghĩa p...,Sai. Cách mạng Tháng Mười đã đánh bại chủ nghĩ...,no,Cách mạng Tháng Mười đã đánh bại biến thể phản...,Cách mạng Tháng Mười đã đánh bại biến thể phản...
2,10fca062-d343-4eca-8434-93c7a8aa5e0e,Vị trí địa lý nằm giữa phương Đông và Địa Trun...,Dựa trên việc Đế quốc Ottoman không bao giờ ki...,Nền ẩm thực của Thổ Nhĩ Kỳ được xây dựng từ gi...,extrinsic,Vị trí địa lý nằm giữa phương Đông và Địa Trun...,Ẩm thực Thổ Nhĩ Kỳ được thiết lập từ giữa thập...
3,ece8eb9e-d6bb-407a-a567-d9531861c603,"Hồ Quý Ly trước có tên là Lê Quý Ly (黎季犛), biể...",Đời cháu thứ 12 của Hồ Hưng Dật sống ở đâu?,"Đời cháu thứ 12 của Hồ Hưng Dật, là Hồ Liêm, s...",no,"Tổ tiên Hồ Quý Ly là Hồ Hưng Dật, vốn là người...","Tổ tiên Hồ Quý Ly là Hồ Hưng Dật, vốn là người..."
4,b613217f-df2a-491e-8326-25811a31eb09,"Năm 2007, Bồ Đào Nha có khoảng 332.137 người n...","Từ thập niên 1990, tại sao xuất hiện một vài l...",Mặc dù Bồ Đào Nha đã đóng cửa biên giới hoàn t...,intrinsic,"Năm 2007, Bồ Đào Nha có khoảng 332.137 người n...","kể từ thập niên 1990, cùng với bùng nổ trong x..."


In [5]:
raw_ds = load_dataset("csv", data_files=CSV_PATH)
ds = raw_ds["train"]
ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'context', 'prompt', 'response', 'label', 'new_context', 'final_context'],
    num_rows: 7000
})

In [6]:
def map_labels(example):
    mapping = {
        "no": "known",
        "extrinsic": "unknown",
        "intrinsic": "known"
    }
    if example["label"] in mapping:
        example["label"] = mapping[example["label"]]
    return example

In [7]:
ds = ds.map(map_labels)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [8]:
ds = ds.remove_columns("id")
ds = ds.class_encode_column(LABEL_COL)

Casting to class labels:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [9]:
# Sau khi đã encode
print(ds.features[LABEL_COL])

# hoặc truy cập riêng ClassLabel
label_feature = ds.features[LABEL_COL]
print("Số lớp:", label_feature.num_classes)
print("Danh sách nhãn:", label_feature.names)

# Ví dụ: mapping index -> tên
for i, name in enumerate(label_feature.names):
    print(f"{i} -> {name}")

ClassLabel(names=['known', 'unknown'])
Số lớp: 2
Danh sách nhãn: ['known', 'unknown']
0 -> known
1 -> unknown


In [10]:
# Tách train/val với stratify_by_column
splits = ds.train_test_split(
    test_size=VAL_SIZE,
    seed=SEED,
    stratify_by_column=LABEL_COL
)
dataset = DatasetDict(
    train=splits["train"],
    val=splits["test"]
)

In [11]:
print(dataset)
for split in ["train", "val"]:
    print(f"\n[{split.upper()}] phân bố nhãn:")
    print(dataset[split].to_pandas()[LABEL_COL].value_counts(normalize=True).sort_index())

DatasetDict({
    train: Dataset({
        features: ['context', 'prompt', 'response', 'label', 'new_context', 'final_context'],
        num_rows: 5600
    })
    val: Dataset({
        features: ['context', 'prompt', 'response', 'label', 'new_context', 'final_context'],
        num_rows: 1400
    })
})

[TRAIN] phân bố nhãn:
label
0    0.670357
1    0.329643
Name: proportion, dtype: float64

[VAL] phân bố nhãn:
label
0    0.670714
1    0.329286
Name: proportion, dtype: float64


In [12]:
model_name = 'Qwen/Qwen3-4B-Instruct-2507'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [13]:
id2label = {
    0: 'known',
    1: 'unknown',
}

label2id = {
    'known': 0,
    'unknown': 1,
}

In [14]:
SYSTEM_PROMPT = """You are an expert AI assistant specializing in information verifiability. Your task is to determine if a given Response can be fully judged (either confirmed or contradicted) based **solely** on the information provided in a Context.

First, think step-by-step. Carefully read the Response and compare it to the information presented in the Context. Your goal is to assess whether the Context contains enough information to make a definitive judgment about the Response's truthfulness.

After your step-by-step analysis, you must classify the Response into one of two categories:

1.  **known**: The truthfulness of the Response (whether it is true or false) can be determined entirely by using the information within the Context. All necessary information is **intrinsic** to the provided text. This applies when the Response is either fully supported by or directly contradicts the Context.
2.  **unknown**: It is impossible to determine whether the Response is true or false based only on the Context. The Response contains information that is not mentioned or cannot be inferred from the Context, requiring **extrinsic** or external knowledge for verification.

Based on your conclusion, you must output ONLY the label: `known` or `unknown`. Do not provide any explanations or additional text.
"""

In [15]:
def tokenize_function(data):
    results = {
        "input_ids": [],
        "labels": [],
        "attention_mask": [],
    }
    for i in range(len(data["context"])):
        context = data["context"][i]
        response = data["response"][i]
        answer = id2label[int(data["label"][i])]
        
        input_messages = [
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": f"""### Bối cảnh: {context}

### Phản hồi: {response}

**Nhiệm vụ:** Dựa vào **Bối cảnh**, hãy xác định xem tính đúng sai của **Phản hồi** có thể được kiểm chứng hoàn toàn từ thông tin có sẵn hay không.

Chỉ trả lời bằng một trong hai nhãn sau: `known` hoặc `unknown`.
"""
            },
        ]

        input_output_messages = input_messages + [{"role": "assistant","content": answer}]

        input_prompt_tokenized = tokenizer.apply_chat_template(conversation=input_messages, return_tensors="pt", add_generation_prompt=True)[0]
        input_output_prompt_tokenized = tokenizer.apply_chat_template(conversation=input_output_messages, return_tensors="pt")[0]

        input_prompt_tokenized = tokenizer.apply_chat_template(conversation=input_messages, return_tensors="pt", add_generation_prompt=True)[0]
        input_output_prompt_tokenized = tokenizer.apply_chat_template(conversation=input_output_messages, return_tensors="pt")[0]
    
        input_ids = input_output_prompt_tokenized
        label_ids = torch.cat([
            torch.full_like(input_prompt_tokenized, fill_value=-100),
            input_output_prompt_tokenized[len(input_prompt_tokenized):]
        ])

        results["input_ids"].append(input_ids)
        results["labels"].append(label_ids)
        results['attention_mask'].append(torch.ones_like(input_ids))
    return results

In [16]:
col_names = dataset['train'].column_names
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=col_names,
    num_proc=os.cpu_count(),
)
tokenized_dataset

Map (num_proc=4):   0%|          | 0/5600 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 5600
    })
    val: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 1400
    })
})

In [17]:
print(tokenized_dataset['train'][0])
print(tokenizer.decode(tokenized_dataset['train'][0]['input_ids'], skip_special_tokens=False))

{'input_ids': [151644, 8948, 198, 2610, 525, 458, 6203, 15235, 17847, 57294, 304, 1995, 2739, 12919, 2897, 13, 4615, 3383, 374, 311, 8253, 421, 264, 2661, 5949, 646, 387, 7225, 44387, 320, 49898, 10774, 476, 22296, 12770, 8, 3118, 3070, 1546, 398, 334, 389, 279, 1995, 3897, 304, 264, 9608, 382, 5338, 11, 1744, 3019, 14319, 29208, 13, 10627, 3641, 1349, 279, 5949, 323, 9429, 432, 311, 279, 1995, 10449, 304, 279, 9608, 13, 4615, 5795, 374, 311, 8552, 3425, 279, 9608, 5610, 3322, 1995, 311, 1281, 264, 44713, 19407, 911, 279, 5949, 594, 8046, 30414, 382, 6025, 697, 3019, 14319, 29208, 6358, 11, 498, 1969, 48129, 279, 5949, 1119, 825, 315, 1378, 11059, 1447, 16, 13, 220, 3070, 5278, 95518, 576, 8046, 30414, 315, 279, 5949, 320, 48764, 432, 374, 830, 476, 895, 8, 646, 387, 10838, 11368, 553, 1667, 279, 1995, 2878, 279, 9608, 13, 2009, 5871, 1995, 374, 3070, 258, 45002, 334, 311, 279, 3897, 1467, 13, 1096, 16790, 979, 279, 5949, 374, 2987, 7225, 7248, 553, 476, 5961, 22296, 29995, 279, 9608, 

## Load model

In [18]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj",],
        bias="none",
        task_type="CAUSAL_LM"
    )

model.config.use_cache = False

model = prepare_model_for_kbit_training(
            model,
            gradient_checkpointing_kwargs={"use_reentrant": False}
        )

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

trainable params: 33,030,144 || all params: 4,055,498,240 || trainable%: 0.8145


In [19]:
from transformers import DataCollatorWithPadding
from typing import Any, Dict, List

class RightPaddingDataCollator(DataCollatorWithPadding):
    """The default data collator pads only inputs, not including the labels."""

    def __init__(self, tokenizer, max_length: int = 256):
        super().__init__(tokenizer, max_length=max_length)

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        input_ids, labels, attention_mask = [], [], []
        max_batch_len = max(len(f["input_ids"]) for f in features)

        for sample in features:
            # Convert to torch tensors
            cur_input_ids = torch.tensor(sample["input_ids"], dtype=torch.long)
            cur_labels = torch.tensor(sample["labels"], dtype=torch.long)
            cur_attention_mask = torch.ones_like(cur_input_ids)

            # Next, we pad the inputs and labels to the maximum length within the batch
            pad_token_id = self.tokenizer.pad_token_id
            padding_length = max_batch_len - len(cur_input_ids)
            cur_input_ids = torch.cat([cur_input_ids, torch.full((padding_length,), fill_value=pad_token_id, dtype=torch.long)])
            cur_labels = torch.cat([cur_labels, torch.full((padding_length,), fill_value=-100, dtype=torch.long)])
            cur_attention_mask = torch.cat([cur_attention_mask, torch.zeros((padding_length,), dtype=torch.long)])

            # Truncate the inputs and labels to the maximum length
            cur_input_ids = cur_input_ids[:max_batch_len]
            cur_labels = cur_labels[:max_batch_len]
            cur_attention_mask = cur_attention_mask[:max_batch_len]

            # Append to the return lists
            input_ids.append(cur_input_ids)
            labels.append(cur_labels)
            attention_mask.append(cur_attention_mask)

        # Return formatted batch.
        return {
            "input_ids": torch.stack(input_ids),
            "labels": torch.stack(labels),
            "attention_mask": torch.stack(attention_mask)
        }

data_collator = RightPaddingDataCollator(tokenizer)

In [20]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [21]:
# def clean_label(text: str) -> str:
#     # Xóa toàn bộ <think>...</think>
#     text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
#     return text.strip()


def clean_label(text: str) -> str:
    # Xoá toàn bộ tag mở/đóng kiểu <...> hoặc </...>
    text = re.sub(r"<[^>]+>", "", text)

    # Xoá khoảng trắng thừa
    text = text.strip()

    # Nếu sau khi xoá, label không khớp với label2id
    if text not in label2id:
        # lấy từ cuối cùng (phòng trường hợp model sinh linh tinh)
        text = text.split()[-1]
    return text

In [22]:
def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # Find the first occurrence of a value other than -100 in each row of labels
    valid_indices = (labels != -100).argmax(axis=1) - 1

    # Pad preds to the maximum length within the batch
    max_length_size = max(labels.shape[1] - valid_indices)
    processed_preds = []
    for i in range(labels.shape[0]):
        # Get the valid part of the prediction
        pred = preds[i, valid_indices[i]:]
        
        # Pad with -100 or truncate to max_length_size
        pred = np.pad(pred, (0, max_length_size - len(pred)), 'constant', constant_values=-100)[:max_length_size]

        processed_preds.append(pred)
    preds = np.array(processed_preds)

    # Replace -100 in the preds as we can't decode them
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    processed_preds = []
    for pred in preds:
        end_pred_idx = np.where(pred == tokenizer.eos_token_id)[0]
        if len(end_pred_idx) > 0:
            end_pred_idx = end_pred_idx[0]
            processed_preds.append(pred[:end_pred_idx])
        else:
            processed_preds.append(pred)

    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(processed_preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    

    # Convert the decoded predictions and labels to label ids
    # int_preds = [label2id.get(p, -1) for p in decoded_preds]
    # int_labels = [label2id[label] for label in decoded_labels]
    int_preds, int_labels = [], []
    for p, l in zip(decoded_preds, decoded_labels):
        l = clean_label(l)
        print(f"Raw: {l}")
        cur_label_id = label2id[l]
        int_labels.append(cur_label_id)
    
        try:
            p = clean_label(p)
            print(f"Predict: {p}")
            cur_pred_id = label2id[p]
        except Exception:
            cur_pred_id = (cur_label_id + 1) % len(label2id)
    
        int_preds.append(cur_pred_id)


    # results = f1_metric.compute(predictions=[0, -1, -1], references=[0, 1, 2], average="macro")
    accuracy_results = accuracy_metric.compute(predictions=int_preds, references=int_labels)
    f1_results = f1_metric.compute(predictions=int_preds, references=int_labels, average="macro")
    precision_results = precision_metric.compute(predictions=int_preds, references=int_labels, average="macro")
    recall_results = recall_metric.compute(predictions=int_preds, references=int_labels, average="macro")

    return {
        **accuracy_results,
        **f1_results,
        **precision_results,
        **recall_results
    }

In [23]:
model.config.pad_token_id = tokenizer.pad_token_id
print("Pad Token ID:", model.config.pad_token_id)

Pad Token ID: 151645


In [24]:
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,

    # thời lượng mỗi phiên
    num_train_epochs=1,                         # train 2 epoch/lần rồi resume
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,              # 2 * 4 = 8 (effective BS)
    eval_accumulation_steps=1,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,                 # tốc độ I/O tốt hơn trên GPU.

    
    # log/eval/save
    logging_dir="./logs",
    logging_steps=20,                           # log vừa phải
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,                         # giữ 3 checkpoint gần nhất
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    # tối ưu hóa
    learning_rate=3e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,                          # ~10–25 warmup steps/phiên
    max_grad_norm=1.0,
    optim="paged_adamw_8bit",
    fp16=True,
    bf16=False,

    # tiện ích cho causal LM + collator tùy biến
    remove_unused_columns=False, # thường đặt False khi dùng collator tuỳ biến/Trainer với causal LM để tránh lược bỏ field cần thiết.
    group_by_length=True,        # gom mẫu có độ dài gần nhau → ít pad → nhanh hơn/ổn định hơn.

    # logging
    report_to="none",
    push_to_hub=False,
    save_safetensors=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
    data_collator=data_collator,
)
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.110000,0.110106,0.878571,0.860981,0.865435,0.857027


Raw: known
Predict: known
Raw: known
Predict: known
Raw: unknown
Predict: unknown
Raw: unknown
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: unknown
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: unknown
Predict: unknown
Raw: known
Predict: known
Raw: unknown
Predict: unknown
Raw: known
Predict: known
Raw: unknown
Predict: known
Raw: unknown
Predict: known
Raw: unknown
Predict: unknown
Raw: known
Predict: known
Raw: unknown
Predict: unknown
Raw: unknown
Predict: unknown
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: known
Raw: unknown
Predict: unknown
Raw: known
Predict: known
Raw: known
Predict: unknown
Raw: unknown
Predict: unknown
Raw: known
Predict: known
Raw: known
Predict: known
Raw: known
Predict: kn

TrainOutput(global_step=350, training_loss=0.16013485993657794, metrics={'train_runtime': 18942.6192, 'train_samples_per_second': 0.296, 'train_steps_per_second': 0.018, 'total_flos': 7.7743158339072e+16, 'train_loss': 0.16013485993657794, 'epoch': 1.0})

In [25]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

login(token=secret_value_0) 

In [26]:
model.save_pretrained("my-lora-adapter")
tokenizer.save_pretrained("my-lora-adapter")

('my-lora-adapter/tokenizer_config.json',
 'my-lora-adapter/special_tokens_map.json',
 'my-lora-adapter/chat_template.jinja',
 'my-lora-adapter/vocab.json',
 'my-lora-adapter/merges.txt',
 'my-lora-adapter/added_tokens.json',
 'my-lora-adapter/tokenizer.json')

In [27]:
# from huggingface_hub import create_repo
# create_repo("mbyhphat/dsc-checkpoint")

In [28]:
from huggingface_hub import upload_folder
upload_folder(
    repo_id="mbyhphat/qwen-dsc", 
    folder_path="/kaggle/working/my-lora-adapter",
    commit_message="extrinsic vs intrinsic/no"
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/mbyhphat/qwen-dsc/commit/95fbf26f957cfa634d4c3fec40b33a3866924b22', commit_message='extrinsic vs intrinsic/no', commit_description='', oid='95fbf26f957cfa634d4c3fec40b33a3866924b22', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mbyhphat/qwen-dsc', endpoint='https://huggingface.co', repo_type='model', repo_id='mbyhphat/qwen-dsc'), pr_revision=None, pr_num=None)

In [29]:
from pathlib import Path

ckpts = sorted(Path("/kaggle/working/output").glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[1]))
last_ckpt = str(ckpts[-1])  # ví dụ ./output/checkpoint-350

upload_folder(
    repo_id="mbyhphat/dsc-checkpoint",
    folder_path=last_ckpt,
    commit_message="extrinsic vs intrinsic/no"
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/mbyhphat/dsc-checkpoint/commit/efed1cbc4fd5d28260a314e3c9185e5eb2b6830e', commit_message='extrinsic vs intrinsic/no', commit_description='', oid='efed1cbc4fd5d28260a314e3c9185e5eb2b6830e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mbyhphat/dsc-checkpoint', endpoint='https://huggingface.co', repo_type='model', repo_id='mbyhphat/dsc-checkpoint'), pr_revision=None, pr_num=None)